In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Dropout, Input
from tensorflow.keras import optimizers, regularizers
from sklearn import preprocessing
from sklearn import decomposition
from sklearn.decomposition import KernelPCA

In [2]:
X1=pd.read_csv('X_N_Autumn_48.csv',header=None)
y=pd.read_csv('Y_N_Autumn_24.csv',header=None)

In [3]:
pca = decomposition.KernelPCA(n_components=16)
X = pca.fit_transform(X1)

In [4]:
X.shape

(2137, 16)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,shuffle=True, random_state=1)

In [6]:
import pennylane as qml
from pennylane.templates import QAOAEmbedding
import pennylane as qml

n_qubits =16

dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]
def circuit(weights, f):
    QAOAEmbedding(features=f, weights=weights, wires=range(2))
    return qml.expval(qml.PauliZ(0))

In [7]:
n_layers = 2
weight_shapes = {"weights": (n_layers, n_qubits)}
print(weight_shapes)

{'weights': (2, 16)}


In [8]:
weights = np.random.random(QAOAEmbedding.shape(n_layers, n_wires=2))
features = np.array([1., 2.])

In [9]:
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)

In [10]:
# Traditional NN (MLP)
clayer_1 = tf.keras.layers.Dense(16, activation='sigmoid')
clayer_2 = tf.keras.layers.Dense(24, activation="linear")
#model = tf.keras.models.Sequential([clayer_0,clayer_1, qlayer, clayer_2])
model = Sequential()
model = tf.keras.models.Sequential([clayer_1,qlayer, clayer_2])
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(opt, loss="mse", metrics=["accuracy"])

In [11]:
import timeit
start_time = timeit.default_timer()
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.25, verbose=2, shuffle=True)
elapsed = timeit.default_timer() - start_time

Epoch 1/100


KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test) 

In [ ]:
from sklearn.metrics import r2_score
r2=r2_score(y_test,y_pred,sample_weight=None, multioutput='uniform_average')

In [ ]:
r2

In [ ]:
elapsed

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model.summary()

In [ ]:
y_pred.shape